In [30]:
import json
import pandas as pd

# Cargar el archivo Excel con los códigos ATC
atc_df = pd.read_excel("../../assets/Tabla_ATC.xlsx")

# Crear los diccionarios a partir del Excel
# Ajustamos los nombres de las columnas según la estructura proporcionada
grupo_anatomico_dict = {}  # Nivel (anatómico)
subgrupo_terapeutico_L2_dict = {}  # Nivel 2 (Subgrupo terapéutico)
subgrupo_terapeutico_L3_dict = {}  # Nivel 3 (Subgrupo terapéutico o farmacológico)
subgrupo_terapeutico_L4_dict = (
    {}
)  # Nivel 4 (Subgrupo terapéutico, farmacológico o químico)
principio_activo_dict = {}  # Nivel 5 (Principio activo)

# Mapear grupos anatómicos (nivel 1)
for _, row in atc_df.drop_duplicates(
    subset=[atc_df.columns[0], atc_df.columns[2]]
).iterrows():
    if pd.notna(row[atc_df.columns[0]]) and pd.notna(row[atc_df.columns[2]]):
        grupo_anatomico_dict[row[atc_df.columns[0]]] = row[atc_df.columns[2]]

# Mapear subgrupos terapéuticos nivel 2
for _, row in atc_df.drop_duplicates(
    subset=[atc_df.columns[3], atc_df.columns[5]]
).iterrows():
    if pd.notna(row[atc_df.columns[3]]) and pd.notna(row[atc_df.columns[5]]):
        subgrupo_terapeutico_L2_dict[row[atc_df.columns[3]]] = row[atc_df.columns[5]]

# Mapear subgrupos terapéuticos nivel 3
for _, row in atc_df.drop_duplicates(
    subset=[atc_df.columns[6], atc_df.columns[8]]
).iterrows():
    if pd.notna(row[atc_df.columns[6]]) and pd.notna(row[atc_df.columns[8]]):
        subgrupo_terapeutico_L3_dict[row[atc_df.columns[6]]] = row[atc_df.columns[8]]

# Mapear subgrupos terapéuticos nivel 4
for _, row in atc_df.drop_duplicates(
    subset=[atc_df.columns[9], atc_df.columns[11]]
).iterrows():
    if pd.notna(row[atc_df.columns[9]]) and pd.notna(row[atc_df.columns[11]]):
        subgrupo_terapeutico_L4_dict[row[atc_df.columns[9]]] = row[atc_df.columns[11]]

# Mapear principio activo nivel 5
for _, row in atc_df.drop_duplicates(
    subset=[atc_df.columns[12], atc_df.columns[13]]
).iterrows():
    if pd.notna(row[atc_df.columns[12]]) and pd.notna(row[atc_df.columns[13]]):
        principio_activo_dict[row[atc_df.columns[12]]] = row[atc_df.columns[13]]


# Función para mapear el código ATC
def mapear_atc(ATC):
    if ATC and isinstance(ATC, str):
        L1 = ATC[0] if len(ATC) >= 1 else None  # Nivel (anatómico)
        L2 = ATC[:3] if len(ATC) >= 3 else None  # Nivel 2 (Subgrupo terapéutico)
        L3 = (
            ATC[:4] if len(ATC) >= 4 else None
        )  # Nivel 3 (Subgrupo terapéutico o farmacológico)
        L4 = (
            ATC[:5] if len(ATC) >= 5 else None
        )  # Nivel 4 (Subgrupo terapéutico, farmacológico o químico)
        L5 = ATC[:7] if len(ATC) >= 7 else None  # Nivel 5 (Principio activo)

        grupo_anat = grupo_anatomico_dict.get(L1, None)
        subgr_ter_L2 = subgrupo_terapeutico_L2_dict.get(L2, None)
        subgr_ter_L3 = subgrupo_terapeutico_L3_dict.get(L3, None)
        subgr_ter_L4 = subgrupo_terapeutico_L4_dict.get(L4, None)
        principio_activo = principio_activo_dict.get(L5, None)

        return (
            L1,
            grupo_anat,
            L2,
            subgr_ter_L2,
            L3,
            subgr_ter_L3,
            L4,
            subgr_ter_L4,
            L5,
            principio_activo,
        )
    else:
        return (
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
        )


# Cargar el archivo JSON con las fichas técnicas
with open(
    "../../data/outputs/2_data_preprocessing/wrangler/medicamentos.json",
    "r",
    encoding="utf-8",
) as f:
    datos_json = json.load(f)

medicamentos_list = []
for nombre_archivo, datos in datos_json.items():
    datos["nombre_archivo"] = nombre_archivo
    medicamentos_list.append(datos)

medicamentos_df = pd.DataFrame(medicamentos_list)
medicamentos_df = medicamentos_df.drop_duplicates()

if medicamentos_df["ATC"].isnull().any():
    print("Hay valores nulos en 'ATC', se van a llenar con 'None'.")

medicamentos_df = medicamentos_df.fillna({"ATC": pd.NA})

mapeo_atc = medicamentos_df["ATC"].apply(mapear_atc)

medicamentos_df[
    [
        "ATC_Nivel_Anatomico",
        "Descripcion_Nivel_Anatomico",
        "ATC_Nivel_2_Subgrupo_Terapeutico",
        "Descripcion_Nivel_2_Subgrupo_Terapeutico",
        "ATC_Nivel_3_Subgrupo_Terapeutico_Farmacologico",
        "Descripcion_Nivel_3_Subgrupo_Terapeutico_Farmacologico",
        "ATC_Nivel_4_Subgrupo_Terapeutico_Farmacologico_Quimico",
        "Descripcion_Nivel_4_Subgrupo_Terapeutico_Farmacologico_Quimico",
        "ATC_Nivel_5_Principio_Activo",
        "Descripcion_Nivel_5_Principio_Activo",
    ]
] = pd.DataFrame(mapeo_atc.tolist(), index=medicamentos_df.index)

print("\nPrimeras filas del DataFrame final con mapeo de ATC:")
display(medicamentos_df.head())

medicamentos_df.to_json(
    "../../data/outputs/2_data_preprocessing/fichas_tecnicas_mapped_atc.json",
    orient="records",
    lines=False,
    force_ascii=False,
    indent=4,
)

print("\nDataFrame guardado correctamente en el archivo JSON.")


Primeras filas del DataFrame final con mapeo de ATC:


,indicaciones,posologia,contraindicaciones,advertencias,interacciones,fertilidad_embarazo,efectos_conducir,reacciones_adversas,sobredosis,ATC,...,ATC_Nivel_Anatomico,Descripcion_Nivel_Anatomico,ATC_Nivel_2_Subgrupo_Terapeutico,Descripcion_Nivel_2_Subgrupo_Terapeutico,ATC_Nivel_3_Subgrupo_Terapeutico_Farmacologico,Descripcion_Nivel_3_Subgrupo_Terapeutico_Farmacologico,ATC_Nivel_4_Subgrupo_Terapeutico_Farmacologico_Quimico,Descripcion_Nivel_4_Subgrupo_Terapeutico_Farmacologico_Quimico,ATC_Nivel_5_Principio_Activo,Descripcion_Nivel_5_Principio_Activo
0,en base a su efecto antiagregante plaquetario ...,posología: como inhibidor de la agregación pla...,no se debe administrar ácido acetilsalicílico ...,dado el efecto antiagregante plaquetario del á...,combinaciones contraindicadas : metotrexato ut...,embarazo el ácido acetilsalicílico atraviesa l...,no se ha observado ningún efecto sobre la capa...,los efectos adversos del ácido acetilsalicílic...,diagnóstico: - los síntomas de intoxicación mo...,B01AC06,...,B,BLOOD AND BLOOD FORMING ORGANS,B01,ANTITHROMBOTIC AGENTS,B01A,ANTITHROMBOTIC AGENTS,B01AC,Platelet aggregation inhibitors excl. heparin,B01AC06,acetylsalicylic acid
1,tratamiento o alivio sintomático del dolor oca...,posología: dosis media recomendada: adultos y ...,no se debe administrar ácido acetilsalicílico ...,dado el efecto antiagregante plaquetario del á...,combinaciones contraindicadas : metotrexato ut...,embarazo el ácido acetilsalicílico atraviesa l...,no se ha observado ningún efecto sobre la capa...,los efectos adversos del ácido acetilsalicílic...,diagnóstico: - los síntomas de intoxicación mo...,N02BA01,...,N,NERVOUS SYSTEM,N02,ANALGESICS,N02B,OTHER ANALGESICS AND ANTIPYRETICS,N02BA,Salicylic acid and derivatives,N02BA01,acetylsalicylic acid
2,alivio de los síntomasde procesos catarrales y...,posología: adultos mayores de 18 años: 1 sobre...,insuficiencia renal y hepática grave. - hipers...,"- debido a su contenido en paracetamol, se deb...",interacciones debidas al paracetamol: el parac...,embarazo: paracetamol: datos epidemiológicos d...,este medicamento puede producir somnolencia al...,durante el periodo de utilización del paraceta...,paracetamol: la sintomatología por sobredosis ...,N02BE51,...,N,NERVOUS SYSTEM,N02,ANALGESICS,N02B,OTHER ANALGESICS AND ANTIPYRETICS,N02BE,Anilides,N02BE51,"paracetamol, combinations excl. psycholeptics"
3,abacavir accord está indicado en el tratamient...,abacavir se debe prescribir por médicos con ex...,hipersensibilidad a abacavir o a alguno de los...,reacciones de hipersensibilidad (ver también s...,el potencial de interacciones mediadas por el ...,"embarazo como norma general, cuando se tome la...",no se han realizado estudios de los efectos so...,en el caso de muchas reacciones adversas comun...,se han administrado dosis únicas de hasta 1.20...,J05AF06,...,J,ANTIINFECTIVES FOR SYSTEMIC USE,J05,ANTIVIRALS FOR SYSTEMIC USE,J05A,DIRECT ACTING ANTIVIRALS,J05AF,Nucleoside and nucleotide reverse transcriptas...,J05AF06,abacavir
4,abacavir/lamivudina accord está indicado en el...,el tratamiento debe ser prescrito por un médic...,hipersensibilidad a los principios activos o a...,se incluyen en este epígrafe las advertencias ...,este medicamento contiene abacavir y lamivudin...,"embarazo como norma general, cuando se decida ...",no se han realizado estudios de los efectos so...,resumen del perfil de seguridad las reacciones...,no se han identificado síntomas o signos espec...,J05AR02,...,J,ANTIINFECTIVES FOR SYSTEMIC USE,J05,ANTIVIRALS FOR SYSTEMIC USE,J05A,DIRECT ACTING ANTIVIRALS,J05AR,"Antivirals for treatment of HIV infections, co...",J05AR02,lamivudine and abacavir



DataFrame guardado correctamente en el archivo JSON.
